In [1]:
import numpy as np
source1 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca.txt"
source2 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca-cls.txt"
source3 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd.txt"
source4 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd-cls.txt"
source5 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c5.txt"
source6 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c10.txt"
source7 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d5.txt"
source8 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d10.txt"
source9 = "F:/MSc DS/Project/dms/trained_dms_txt/word2dm.txt"

# dim = 17 #

def get_dict_from_source(source = source5,dim =17):
    if source ==source1 or source==source3:
        dim = 15
    whole =[]

    with open(source) as f:
        for line in f:
            linelist = line.split()
            for a in linelist:
                whole.append(a)
    chunk = dim**2+1
    n = len(whole)/chunk
    voc = []
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        a = whole[i * chunk + 1:(i + 1) * chunk]
        a = np.array(a, dtype='float')
        a.resize([dim, dim])
        dic[voc[i]] = a

    return dic

def ke(a, b):
    dim = a.shape[0]
    m = b-a
    eig,_ = np.linalg.eig(m)
    n =len(eig)
    for i in range(n):
        if eig[i] >=0:
            eig[i]=0
        else:
            eig[i]=-eig[i]
    e= np.eye(dim,dtype= float)
    for i in range(dim):
        e[i][i]=eig[i]
    
    Ke = 1-np.linalg.norm(e)/np.linalg.norm(a)
    return Ke

def halfmatrix(m):
    eig,vec = np.linalg.eig(m)
    n = len(eig)
    for i in range(n):
        if eig[i]<0:
            eig[i]=0
    eignew = eig**0.5
    dim = m.shape[0]
    e = np.eye(dim)
    for i in range(dim):
        e[i][i] =eignew[i]
    vec_T = np.linalg.inv(vec)
    half = vec@e@vec_T
    return half

def compute_Ke(listA,listB, dic, method ='add'):
    '''
    imput two phrase or sentences lists
    tokenize each pair and get new dm using composition method
    compute Ke pred for each pair
    if oov set Ke to oov  # after test all in voc

    composition method
    add or pointwise
    '''

    n = len(listA)
    if method == 'add':
        Ke_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] + dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()] + dic[phraseB[1].lower()]
            Ke_list.append(ke(newA, newB))

    if method == 'pointwise':
        Ke_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] * dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()] * dic[phraseB[1].lower()]
            Ke_list.append(ke(newA, newB))
            
    if method == 'dot':
        Ke_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()]@dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()]@dic[phraseB[1].lower()]
            Ke_list.append(ke(newA, newB))
    
    if method == 'mult':
        Ke_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            v1 = dic[phraseA[0].lower()]
            n1 = dic[phraseA[1].lower()]
            v2 = dic[phraseB[0].lower()]
            n2 = dic[phraseB[1].lower()]
            newA = halfmatrix(n1)*v1*halfmatrix(n1)
            newB = halfmatrix(n2)*v2*halfmatrix(n2)
            Ke_list.append(ke(newA, newB))
            
    return Ke_list


# get sv dataset
dicA = []
dicB = []
val = []
with open("F:/MSc DS/Project/compositional datasets/KS2016-VO.txt") as f:
    for line in f:
        linelist = line.split(",")
        dicA.append(linelist[0])
        dicB.append(linelist[1])
        val.append(linelist[2])

# convert the T to 1 F to 0 in val list
n = len(val)
for i in range(n):
    if val[i]=='T\n':
        val[i]=1
    else:
        val[i]=0
# dicA  list to store the first phrase
# dicB  list to store the second pharse
# val  true val




In [2]:
# remove oov
dicA.pop(254)
dicB.pop(254)
val.pop(254)


1

In [3]:
dicA.pop(254)
dicB.pop(254)
val.pop(254)

0

In [4]:
dic1 = get_dict_from_source(source1)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.44849115504682624
0.4764169976002888


In [5]:
dic1 = get_dict_from_source(source2)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.4933848669540657
0.5385758882116842


In [6]:
dic1 = get_dict_from_source(source3)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.49397948565482386
0.6045148548493279


In [7]:
dic1 = get_dict_from_source(source4)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.5034509121026142
0.5207160908067702


In [8]:
dic1 = get_dict_from_source(source5)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.49109133767971286
0.4418866401919769


In [9]:
dic1 = get_dict_from_source(source6)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.4797086368366286
0.46055342011934847


In [10]:
dic1 = get_dict_from_source(source7)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.4728068126314001
0.41948225700269703


In [11]:
dic1 = get_dict_from_source(source8)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.45651850750706113
0.39828834759710335


In [12]:
dic1 = get_dict_from_source(source9)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.49701628830512423
0.49463781350209174
